Agenda
- 各自讲讲模型部分（怎么建的、有啥问题）
- 一起讨论大的目标


TODO
- all: 顺着大方向，作一些调整，补充markdown解释
- jinming: try implict ALS CF Model. Use MAE?
- leyi: 改reco，加roc-based metrics, evaluate model 2

ps: 避免漏掉修改，两个人合一份，最后再把两份整合到一起

# 1. Business Objectives

Imagine that you work for a digital media company that needs to recommend movies. What is your objective? What are you trying to optimizing and what are you willing to sacrifice? 

State your business objectives in building a recommendation system. Indicate

- Objective function: state your business objective, and how this would translate to the metrics that you will ultimately be optimizing for	 	 	 		
- Any metric you wish to optimize in addition to accuracy 
- The intended user （active users with certain amount of rating records）
- Any business rules you think will be important  （推荐内容的种类多样性？不重复性？但比较复杂，这次作业可能暂时实现不出来）
- Performance requirements (time and/or complexity) （训练和预测时间要可接受，支持每天更新，支持大样本）
- And more (for example: interpretability, diversity, novelty, etc) 		

初步想法：

目标：首页推荐TOP K电影给每个用户

实现：训练模型（min mae） -> 推荐电影给用户（max recall/auc, user_coverage)

- Based on the movie-rating historical data, we are to provide personalizied lists of movies specific to active user, in section like "TOP 10 MOVIES THAT YOU MAY ALSO LIKE" in the APP.

- To goal is to make good recommendations to active users, in order to increase the chance of them clicking into, watching and liking one of the recommended movies. 

- Therefore, while maximizing rating accuracy for model training, our ultimate goal is to improve recall/AUC and user-coverage: the higher these metrics are, the better job we are doing in making well recommendations to active users. 

- To achieve this goal, we will limit our training set to only include movies and users with certain number of rating records, so that we could achieve better overall accuracy thus giving better recommendations. 

- Inevitably, this will limit the system's performance in cold-start recommendations to some extent but this is acceptable under this overall objective. So we might care less on metrics like movie-coverage, catalog-coverage.

reco func可能要完善一下：推荐之前没看过的（就是没在train set里的）。更严格来讲，应该推荐某个日期之后的。但实现起来可能比较复杂。

# 2. Model Exploration & Evaluation 

In [1]:
# import packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time
from sklearn.model_selection import train_test_split
from item_based_cf import *
from metrics_helper import *

In [2]:
df = pd.read_csv('./ml-latest/ratings.csv')
df.drop("timestamp", inplace=True, axis=1)

In [ ]:
n_users = df['userId'].nunique()
n_items = df['movieId'].nunique()
sparsity = round(1.0-len(df)/float(n_users*n_items),5)
print(f'The number of unique users is {n_users}')
print(f'The number of unique items is {n_items}')
print(f'The sparsity is {sparsity*100}%')

fig, ax = plt.subplots(figsize=(12,6))
fig = sns.histplot(df['movieId'])
ax.set_ylabel('Percentage of Users Rating the Movie', size=18)
ax.set_xlabel('MovieID', size=18)
ax.set_title('Ratings Distribution by Movies', size=20)
plt.show()

cover movies of all popularity level: set some lower threshold for movies, like only include movies with greater than 5 or 100 ratings, but I wouldn't select only the most popular movies.

If you include all levels of popularity it would be great to subset your accuracy results by popularity. Do you see a curve in accuracy from least to most popular?

sample from sampled dataset


In [28]:
#LEYI:能不能这样，先不设定门槛，随机sample，这样可以包括所有popularity程度的电影。
#然后将popularity分大概3层或者5层，分别看看每个层级的acc如何（例如一个柱状图or curve），最后结合这个结果决定
#我们认为至少需要多少程度的电影，才能够获得比较好的准确度，所以暂时作出牺牲，不推荐那些相对太冷的电影给用户。
#因为追求的是“reco the right movies to users"，而不是要保证cover到所有popularity程度的电影

## 2.1 Memory-based Collaborative Filtering

### 2.1.1 Data preprocessing

In [11]:
##default param is 4300 and 6800, which can satisfy hw requirement, and it takes about 6 hours to train and predict
df_sample = sample_df(df, min_item=20, min_user=1000)

number of users: 169553
number of items: 3929
shape of sampled df: (23641130, 3)


In [27]:
# LEYI: 可能需要稍微解释下为啥是这个thresholds？
# 体感这个标准还是有点略严格，只包括评过2000+部电影的用户、被10000+个用户评过的电影，似乎有点冷启动不友好？

In [9]:
df_sample.head()

,userId,movieId,rating
128418,1272,1,3.5
128419,1272,7,2.0
128420,1272,11,4.0
128421,1272,16,2.5
128422,1272,21,3.0


In [10]:
train_set, test_set = train_test_split(df_sample, test_size=0.50)
# train_set, valid_set = train_test_split(train_set, test_size=0.2)
rating_matrix_train = train_set.pivot(index='movieId', columns='userId', values='rating')
rating_matrix_test = test_set.pivot(index='movieId', columns='userId', values='rating')

In [53]:
train_small = rating_matrix_train.iloc[:100,:100]

In [74]:
test_small = rating_matrix_test.iloc[:100,:100]

### 2.1.2 Model Training 

In [79]:
class ItemBasedCF():
    def __init__(self, k_neighbors = 10, similarity_metrics = 'pearson'):
        self.train_set = None
        self.similarity_matrix = None
        self.k_neighbors = k_neighbors
        self.similarity_metrics = similarity_metrics

    def fit(self, train_df, dummy_y=None):
        train_df = train_set.pivot(index='movieId', columns='userId', values='rating')#修改输入格式
        
        self.train_set = train_df.values
        self.similarity_matrix = sim_matrix(self.train_set, self.similarity_metrics)

    def predict(self, test_df):
        test_df = test_df.pivot(index='movieId', columns='userId', values='rating')
        
        pred_df = np.zeros(test_df.shape)
        train_set_filled = np.nan_to_num(self.train_set)
        nbrs = NearestNeighbors(n_neighbors=self.k_neighbors+1, algorithm='ball_tree').fit(train_set_filled)
        distances, indices = nbrs.kneighbors(train_set_filled)
        
        # return average score of each movie with NAN ignored
        mean_item = np.nanmean(self.train_set, axis=1)
        for i in range(len(pred_df)):
            pred_df[i][:] = mean_item[i]
            neighbor_indice = indices[i][1:]
            for j in range(len(pred_df[i])):
                if np.isnan(self.train_set[i][j]):
                    pred_df[i][j] += np.nansum(self.similarity_matrix[i][neighbor_indice] * (
                                self.train_set[:, j][neighbor_indice] - mean_item[neighbor_indice])) / np.sum(
                        self.similarity_matrix[i][neighbor_indice])
                else:
                    pred_df[i][j] = np.nan
        return pred_df
    
    # requested methods for sk-learn estimator
    def get_params(self, deep = True):
        return {"k_neighbors": self.k_neighbors, "similarity_metrics": self.similarity_metrics}

    def set_params(self, **parameters):
        for parameter, value in parameters.items():
            setattr(self, parameter, value)
        return self

In [72]:
#LEYI: put the adjusted ItemBasedCF to the python file?

In [70]:
model1 = ItemBasedCF(5, 'pearson')

In [71]:
start = time.time()
model1.fit(rating_matrix_train)
# model1.fit(train_small)
end = time.time()
print(f'training_time in seconds: {end-start}')

training_time in seconds: 356.51662373542786


### 2.1.3 Model Tuning with Cross validation

In [64]:
model1.train_set

In [65]:
#LEYI: save the models' final weights to local, for greater speed?

In [80]:
start = time.time()
pred_set = model1.predict(rating_matrix_test)
# pred_set = model1.predict(test_small)
end = time.time()
print(f'training_time in seconds: {end-start}')

In [138]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer

LEYI: Reasons to use MAE:

One large characteristic of Mean Average Error(MAE) does not give any bias to extrema in error terms. If there are outliers or large error terms, it will weigh those equally to the other predictions. Therefore, MAE should be preferred when looking toward rating accuracy when you’re not really looking toward the importance of outliers. To get a holistic view or representation of the Recommender System, use MAE.

https://towardsdatascience.com/evaluating-recommender-systems-root-means-squared-error-or-mean-absolute-error-1744abc2beac

In [139]:
# Reform loss function for gridsearch
def My_MAE(pred, test):
    try:
        test = test.pivot(index='movieId', columns='userId', values='rating')
    except:
        pass
    try: 
        pred = pred.pivot(index='movieId', columns='userId', values='rating')
    except:
        pass
    return MAE(pred, test)
scoring = make_scorer(MAE, greater_is_better = False)

In [140]:
parameters = {'similarity_metrics':['pearson','cosine'], 'k_neighbors':np.arange(10,35,5)}
model_test = ItemBasedCF()
gs = GridSearchCV(model_test, parameters, scoring = scoring, cv=3)

In [142]:
gs.fit(df_sample, df_sample) #改成train set

In [136]:
print(gs.best_params_)

{'k_neighbors': 10, 'similarity_metrics': 'pearson'}


In [135]:
print(gs.best_score_)

0.8884583973284366


In [21]:
#LEYI: add some markdown here? 简单说说最后模型用了啥param，最优结果是多少

### 2.1.4 Model Evaluation

How do the evaluation metrics change as a function of your model size? Systematically change your data set size by again sampling your data from a small size to a large size (25%, 50%, 75%, 100%):
1. Does overall accuracy change?
2. What about the distribution of accuracy over users or items?
3. How does run-time scale with data size?

In [24]:
#TODO: ADD PLOTS TO answer:
#How does the evauation metrics change as sample size changes? like 25%, 50%, 75%, 100%
#How does the training and testing runtime change as sample size changes? like 25%, 50%, 75%, 100%

#### (1) Accuracy

##### error-based accuracy

In [21]:
MAE(rating_matrix_test, pred_set)

0.6885897602816633

In [22]:
MSE(rating_matrix_test, pred_set)

0.7899858612161919

In [82]:
from sklearn.metrics import recall_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_score


def recall(test, pred): #test, pred是2维矩阵
    return recall_score(test.reshape(-1), pred.reshape(-1))    

def auc(test, pred):
    return roc_auc_score(test.reshape(-1), pred.reshape(-1))

def precision(test, pred):
    return precision_score(test.reshape(-1), pred.reshape(-1))

#测试可以用之后，放进metrics.py

##### ranking-based accuracy

In [29]:
dcg(rating_matrix_test.values, pred_set)

22.91174836018709

In [ ]:
#TOADD: ndcg

#### (2) Coverage

In [ ]:
# user_coverage: the fraction of users for which at least k items can be recommended well
user_coverage = cal_coverage(test, pred_set, threshold=3, coverage_type = 'user')

# item_coverage: the fraction of items that can be recommended to at least k users well
item_coverage = cal_coverage(test, pred_set, threshold=3, coverage_type = 'item')

# catalog_coverage: the fraction of items that are in the top-k for at least 1 user
catalog_coverage = cal_coverage(test, pred_set, coverage_type = 'catalog')

#### (3)Scalability

In [25]:
#add plots: runtime vs sample size 

### 2.1.5 Reflection

- Other design choices to consider?

## 2.2 Model-based  Collaborative Filtering

### 2.2.1 Data preprocessing

In [1]:
#initialize spark 
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("PySpark ML Basics").getOrCreate()

#process data into dataframe format

df = spark.read.format('csv').options(header='true').load('./ml-latest/ratings.csv').toDF("userId",  "movieId",  "rating",   "timestamp")

from pyspark.sql.functions import *
from pyspark.sql.types import *


df = df.withColumn("userId", df["userId"].cast(IntegerType())). \
withColumn("movieId", df["movieId"].cast(IntegerType())). \
withColumn("rating",df["rating"].cast(FloatType())). \
withColumn("timestamp", df['timestamp'].cast(LongType()))

In [2]:
df.printSchema()

root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: float (nullable = true)
 |-- timestamp: long (nullable = true)



In [2]:
def sample_df_pyspark(df, min_item=20, min_user=1000):
    
    users = df.groupby('userId').count()
    users= users.withColumn('num_of_movies',users['count']).select('userId','num_of_movies')
    
    movies = df.groupby('movieId').count()
    movies = movies.withColumn('num_of_users',movies['count']).select('movieId','num_of_users')
    
    tmp = movies.filter(movies.num_of_users>=min_user).\
                join(df,df.movieId==movies.movieId).\
                select(df.userId,df.movieId,df.rating,df.timestamp)
    sample = users.filter(users.num_of_movies>=min_item).\
                join(tmp,tmp.userId==users.userId).\
                select(tmp.userId,tmp.movieId,tmp.rating,tmp.timestamp)

    return sample 

In [3]:
df_sample = sample_df_pyspark(df, min_item=200, min_user=5000)
df_sample.printSchema()

root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: float (nullable = true)
 |-- timestamp: long (nullable = true)



In [4]:
#train test split 
train,test=df_sample.orderBy(rand()).randomSplit((0.8,0.2),seed=1234)

#or read train/test data

### 2.2.2 Model Training

#### basic model

In [5]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS

als = ALS(maxIter=5, regParam=0.01, userCol="userId", itemCol="movieId", ratingCol="rating",
          coldStartStrategy="drop")
model = als.fit(train)

# Evaluate the model by computing the RMSE on the test data
predictions = model.transform(test)
evaluator = RegressionEvaluator(metricName="mae", labelCol="rating",
                                predictionCol="prediction")
#改下MAE

mae = evaluator.evaluate(predictions)
print("mean absolute error = " + str(mae))


mean absolute error = 0.6075575849407786


In [7]:
predictions.filter(predictions.userId==10362).show()

+------+-------+------+---------+----------+
|userId|movieId|rating|timestamp|prediction|
+------+-------+------+---------+----------+
| 10362|    471|   4.0|946845996| 3.4707136|
| 10362|   1223|   4.0|946847119|  4.091524|
| 10362|   1199|   3.0|946847985|   3.86697|
| 10362|     34|   4.0|946845484|  4.084771|
| 10362|   1198|   4.0|946848947| 4.1369057|
| 10362|   1183|   3.0|946849928| 3.1311548|
| 10362|    916|   4.0|946845664|  4.024228|
| 10362|   2804|   4.0|946845452| 4.0380464|
| 10362|   1208|   4.0|946849687| 3.7933345|
| 10362|   1961|   3.0|946850183|   3.36571|
| 10362|    539|   4.0|946847485| 2.9690156|
| 10362|   1197|   5.0|946845620|  4.110145|
| 10362|   2628|   3.0|946848528|  2.067631|
| 10362|   1207|   4.0|946849507|  4.278883|
| 10362|   2109|   4.0|946846291| 3.5137308|
| 10362|   2193|   4.0|946848506|  2.702454|
| 10362|    306|   4.0|946851003|  3.903015|
| 10362|   1917|   2.0|946849223| 1.6949416|
| 10362|   2485|   4.0|946846614| 2.3564868|
| 10362|  

In [39]:
#LEYI: can try the implicit ALS model
#als = ALS(maxIter=5, regParam=0.01, implicitPrefs=True,
#           userCol="userId", itemCol="movieId", ratingCol="rating")

### 2.2.3 Model Tuning with Cross validation

In [41]:
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder 


paramGrid = ParamGridBuilder() \
    .addGrid(als.maxIter,[5,10,20]) \
    .addGrid(als.regParam, [0.1, 0.01,0.001]) \
    .build()

crossval = CrossValidator(estimator=als,
                          estimatorParamMaps=paramGrid,
                          evaluator=RegressionEvaluator(metricName="rmse", labelCol="rating",
                                predictionCol="prediction"),
                          numFolds=5)  # use 3+ folds in practice

# Run cross-validation, and choose the best set of parameters.
cvModel = crossval.fit(train)


# Make predictions on test documents. cvModel uses the best model found (lrModel).
prediction = cvModel.transform(test)

In [45]:
paramMaps = cvModel.getEstimatorParamMaps()

In [74]:
results = np.array(cvModel.avgMetrics)
bestparams = paramMaps[np.argmin(results)]
print("Best Params:", bestparams)
print("The miniest rmse:", np.min(results)) 

Best Params: {Param(parent='ALS_1020de5001ff', name='maxIter', doc='max number of iterations (>= 0).'): 20, Param(parent='ALS_1020de5001ff', name='regParam', doc='regularization parameter (>= 0).'): 0.001}
The miniest rmse: 0.7546022373461236


In [79]:
#save best model 
model_path = "./model"
cvModel.write().save(model_path)

### 2.2.4 Model Evaluation

- ROC-based metrics (recall, precision, roc curve, auc)
- DCG
- coverage
- scalability

In [25]:
#sample users to evaluate 
test.createOrReplaceTempView("test")
predictions.createOrReplaceTempView("predictions")
sample_users = spark.sql("select distinct userId from test limit 1000")
sample_users.createOrReplaceTempView("sampleuser") 

In [62]:
# from pyspark.sql.window import Window   

# #get top 10 recommendations for sample users 
# userRecs = model.recommendForUserSubset(sample_users,10)
# cleanedRecs=userRecs.select('userId',explode(userRecs.recommendations.movieId).alias("movieId")).withColumn("y",lit(1)) 


# #get top 10 rated movies of sample users 
# toprank = test.select("userId","movieId","rating", row_number().\
#                        over(Window.partitionBy("userId").\
#                             orderBy(desc("rating"))).alias("rank")).withColumn("y",lit(1))
# toprank.createOrReplaceTempView("toprank")

# useRats = spark.sql("select userId, movieId, y from toprank where userId in (select userId from sampleuser) and rank<=10")

In [10]:
num_users= 1000
num_items= df_sample.select('movieId').distinct().count() 
from scipy import sparse
import numpy as np

#Construct a lookup map (movieId to Long) 
movie_key_map = df_sample.select('movieId').distinct().rdd.map(lambda x:x[0]).zipWithIndex().collectAsMap()

#Construct a lookup map (movieId to Long) 
user_key_map = df_sample.select('userId').rdd.map(lambda x:x[0]).zipWithIndex().collectAsMap()

In [12]:
def recodf2array(df,valcol,num_users=num_users,num_items=num_items): 
    
    rows = np.concatenate(
            df.select("userId").rdd.glom().map(
              lambda x: np.array([user_key_map.get(elem[0]) for elem in x]))
            .collect())
    
    cols = np.concatenate(
            df.select("movieId").rdd.glom().map(
              lambda x: np.array([movie_key_map.get(elem[0]) for elem in x]))
            .collect())
    
    data = np.concatenate(
            df.select(valcol).rdd.glom().map(
              lambda x: np.array([elem[0] for elem in x]))
            .collect())
    
    sparse_matrix = sparse.coo_matrix((data, (rows, cols)), 
                        shape=(num_users, num_items))
    
    sparse_matrix = sparse_matrix.tocsr() 
    
    return sparse_matrix.toarray() 

In [27]:
#transform dataframe to matrix 
sub_test = spark.sql("select userId,movieId,rating from test where userId in (select userId from sampleuser)")
sub_predictions = spark.sql("select userId,movieId,prediction from predictions where userId in (select userId from sampleuser)")
test_matrix = recodf2array(sub_test,'rating')
pred_matrix = recodf2array(sub_predictions,'prediction')

- dcg & ndcg 

In [34]:
print("dcg:", dcg(test_matrix,pred_matrix))
print("ndcg:", ndcg(test_matrix,pred_matrix))

dcg: 165.43502031840197
ndcg: 1.0


- coverage 

In [40]:
print("user coverage:", cal_coverage(test_matrix, pred_matrix, k_items=10, threshold=3, coverage_type='user')) 

user coverage: 0.9285714285714286


- ROC based metrix

In [39]:
print(roc_metrics(test_matrix, pred_matrix, 10))

(0.5382440476190476, 0.766789923039923, 0.5382440476190476)


### 2.2.5 Reflection

- Other design choices to consider?

# 3. Final Recommendation

• How does your recommendation system meet your hypothetical objectives? Would you feel comfortable putting these solutions into production at a real company? What would be the potential watch outs?

• What is your final recommendation to your manager?

- Final recommendation to your manager
- Reasons and potential watch outs 

In [ ]:
#举例